##                                           NLP Applications Assignment 1

##                                                       Group 80

##                                               Question Answering System

In [17]:

### Group Members Name with Student ID:#
from IPython.display import display, HTML
html_content = """

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">

    <style>
        .table {
            display: table;
            width: 70%;
            border-collapse: collapse;
        }
        .table-row {
            display: table-row;
        }
        .table-cell {
            display: table-cell;
            border: 1px solid #dddddd;
            padding: 8px;
        }
        .header {
            background-color: #fffff;
            font-weight: bold;
            font-size: 100%;
        }
    </style>
</head>
<body>
 <h3>Group Members Name with Student ID:</h3>

<div class="table" style="float: left">

    <div class="table-row header">
        <div class="table-cell">BITS ID</div>
        <div class="table-cell">Name</div>
        <div class="table-cell">BITS ID</div>
        <div class="table-cell">Email ID</div>
        <div class="table-cell">Contribution</div>
    </div>
    <div class="table-row">
        <div class="table-cell">1.</div>
        <div class="table-cell">GANESHKUMAR KARUPPAIAH</div>
        <div class="table-cell">2023aa05600</div>
        <div class="table-cell">2023aa05600@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">100%</div>
    </div>
    <div class="table-row">
    <div class="table-cell">2.</div>
        <div class="table-cell">KOLLURI VENKATESWARA SWAROOP</div>
        <div class="table-cell">2023aa05945</div>
        <div class="table-cell">2023aa05945@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">100%</div>
    </div>
    <div class="table-row">
    <div class="table-cell">3.</div>
        <div class="table-cell">RAJESH J</div>
        <div class="table-cell">2023aa05859</div>
        <div class="table-cell">2023aa05859@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">100%</div>
    </div>
    <div class="table-row">
    <div class="table-cell">4.</div>
        <div class="table-cell">VIJAYA LAKSHMI R</div>
        <div class="table-cell">2023aa05341</div>
        <div class="table-cell">2023aa05341@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">100%</div>
    </div>
    <div class="table-cell">5.</div>
        <div class="table-cell">BHALANI AMIT KISHORKUMAR</div>
        <div class="table-cell">2023ab05169</div>
        <div class="table-cell">2023ab05169@wilp.bits-pilani.ac.in</div>
        <div class="table-cell">100%</div>
    </div>
</div>
</html>
"""
display(HTML(html_content))

# Import Required Libraries
Import the necessary libraries, including transformers, torch, numpy, pandas, and matplotlib.

In [1]:
# Import Required Libraries

# Import the necessary libraries for the notebook
import transformers  # For BERT model and tokenization
import torch  # For tensor operations and model training
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation and analysis
import matplotlib.pyplot as plt  # For plotting graphs

# Print the versions of the libraries to ensure compatibility
print("Transformers version:", transformers.__version__)
print("Torch version:", torch.__version__)
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)

Transformers version: 4.47.1
Torch version: 2.5.1+cpu
Numpy version: 1.26.4
Pandas version: 2.2.2


# Import Dataset and Extract Features and Labels
Load the dataset from the provided URL, extract features and labels, and take a sample of 100 data points for initial processing.

In [2]:
# Import Dataset and Extract Features and Labels

import json
import requests

# Load the dataset from the provided URL
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
response = requests.get(url)
data = response.json()

# Extract features and labels from the dataset
contexts = []
questions = []
answers = []

for article in data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            if 'answers' in qa and len(qa['answers']) > 0:
                answer = qa['answers'][0]['text']
                contexts.append(context)
                questions.append(question)
                answers.append(answer)

# Take a sample of 100 data points for initial processing
sample_size = 100
contexts_sample = contexts[:sample_size]
questions_sample = questions[:sample_size]
answers_sample = answers[:sample_size]

# Print the first 5 samples to verify
for i in range(5):
    print(f"Context {i+1}: {contexts_sample[i]}")
    print(f"Question {i+1}: {questions_sample[i]}")
    print(f"Answer {i+1}: {answers_sample[i]}")
    print()

Context 1: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Question 1: When did Beyonce start becoming popular?
Answer 1: in the late 1990s

Context 2: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in va

# Vectorize Features and Labels using Sentence Embedding Techniques
Use sentence embedding techniques like sentence2vec or doc2vec to vectorize the features and labels. Choose the best embedding technique.

In [3]:
# Vectorize Features and Labels using Sentence Embedding Techniques

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split

# Combine contexts and questions for embedding
combined_texts = [context + " " + question for context, question in zip(contexts_sample, questions_sample)]

# Tag documents for Doc2Vec
tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(combined_texts)]

# Initialize and train the Doc2Vec model
model = Doc2Vec(vector_size=100, window=2, min_count=1, workers=4, epochs=40)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Vectorize contexts and questions
context_vectors = [model.infer_vector(context.split()) for context in contexts_sample]
question_vectors = [model.infer_vector(question.split()) for question in questions_sample]

# Combine context and question vectors
features = [np.concatenate((context_vector, question_vector)) for context_vector, question_vector in zip(context_vectors, question_vectors)]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, answers_sample, test_size=0.2, random_state=42)

# Print the shapes of the training and validation sets
print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))

Training set size: 80
Validation set size: 20


# Fine-tune BERT Model
Fine-tune the BERT model using the vectorized features and labels from the embeddings defined in the previous step.

In [4]:
# Fine-tune BERT Model

from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the input data
input_ids = []
attention_masks = []

for context, question in zip(contexts_sample, questions_sample):
    encoded_dict = tokenizer.encode_plus(
        question, context,
        add_special_tokens=True,
        max_length=512,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert lists to tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Convert answers to tensors
answer_starts = [context.find(answer) for context, answer in zip(contexts_sample, answers_sample)]
answer_ends = [start + len(answer) for start, answer in zip(answer_starts, answers_sample)]
answer_starts = torch.tensor(answer_starts)
answer_ends = torch.tensor(answer_ends)

# Create DataLoader for training and validation sets
dataset = TensorDataset(input_ids, attention_masks, answer_starts, answer_ends)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Load the BERT model for question answering
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
model.train()

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
epochs = 3
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        b_input_ids, b_attention_masks, b_start_positions, b_end_positions = batch

        model.zero_grad()

        outputs = model(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_attention_masks,
            start_positions=b_start_positions,
            end_positions=b_end_positions
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Loss: {avg_train_loss}")

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_bert')
tokenizer.save_pretrained('./fine_tuned_bert')

# Print answers for 5 context and questions
model.eval()
for i in range(5):
    context = contexts_sample[i]
    question = questions_sample[i]

    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = model(input_ids, attention_mask=attention_mask)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    answer = ' '.join(all_tokens[torch.argmax(start_scores): torch.argmax(end_scores) + 1])

    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Ganesh AI\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the mo

Epoch 1, Loss: 6.247320508956909
Epoch 2, Loss: 5.96860990524292
Epoch 3, Loss: 5.774764966964722
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Question: When did Beyonce start becoming popular?
Answer: 

Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record pro

# Model Inference: Answering Questions
The model should take a question and context as input and provide answers. Print answers for 5 contexts and questions.

In [7]:
# Model Inference: Answering Questions

# Load the fine-tuned BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained('./fine_tuned_bert')
tokenizer = BertTokenizer.from_pretrained('./fine_tuned_bert')

# Set the model to evaluation mode
model.eval()

# Define the contexts and questions for inference
contexts_inference = [
    "Statistics is the discipline that concerns the collection, organization, analysis, interpretation, and presentation of data.",
    "Descriptive statistics is a branch of statistics that summarizes and describes the features of a dataset.",
    "Inferential statistics is used to draw conclusions and make predictions about a population based on a sample of data.",
    "Probability theory is a branch of mathematics concerned with the analysis of random phenomena.",
    "A statistical hypothesis is a conjecture about a population parameter that can be tested using statistical methods."
]

questions_inference = [
    "What is statistics?",
    "Where can descriptive statistics be used?",
    "How to draw meaningful conclusions?",
    "What is probability theory?",
    "What is a statistical hypothesis?"
]

# Print answers for the given contexts and questions
for context, question in zip(contexts_inference, questions_inference):
    # Tokenize the input context and question
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Get the model's output
    outputs = model(input_ids, attention_mask=attention_mask)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Find the tokens with the highest start and end scores
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1

    # Convert the tokens to the answer string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))

    # Print the context, question, and answer
    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print()

Context: Statistics is the discipline that concerns the collection, organization, analysis, interpretation, and presentation of data.
Question: What is statistics?
Answer: [CLS] what is statistics ? [SEP] statistics is the discipline that concerns the collection , organization , analysis , interpretation , and presentation of data .

Context: Descriptive statistics is a branch of statistics that summarizes and describes the features of a dataset.
Question: Where can descriptive statistics be used?
Answer: [CLS] where can descriptive statistics be used ? [SEP] descriptive statistics is a branch of statistics that summarizes and describes the features of a dataset .

Context: Inferential statistics is used to draw conclusions and make predictions about a population based on a sample of data.
Question: How to draw meaningful conclusions?
Answer: ##ferential statistics is used to draw conclusions and make predictions about a population based on a sample of data .

Context: Probability theo

# Evaluate the Model
Use the provided evaluation script to evaluate the fine-tuned model. Provide context from the Wikipedia link and ask the specified questions.

In [13]:
# Evaluate the Model
import os
import requests

# Download the evaluation script
evaluation_script_url = "https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/"
evaluation_script_path = "evaluate-v2.0.py"



# Check if the file exists locally
if not os.path.exists(evaluation_script_path):
    print(f"{evaluation_script_path} not found. Downloading from {file_url}...")
    response = requests.get(evaluation_script_url)
    with open(evaluation_script_path, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {evaluation_script_path}.")
else:
    print(f"{evaluation_script_path} already exists.")


# Save the predictions in the required format
predictions = {}
for i, (context, question) in enumerate(zip(contexts_sample, questions_sample)):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = model(input_ids, attention_mask=attention_mask)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))
    predictions[str(i)] = answer

# Save predictions to a JSON file
predictions_path = "predictions.json"
with open(predictions_path, 'w') as f:
    json.dump(predictions, f)

# Save the ground truth answers in the required format
ground_truths = {}
for i, answer in enumerate(answers_sample):
    ground_truths[str(i)] = answer

# Save ground truths to a JSON file
ground_truths_path = "ground_truths.json"
with open(ground_truths_path, 'w') as f:
    json.dump(ground_truths, f)

# Run the evaluation script
subprocess.run(["python", evaluation_script_path, ground_truths_path, predictions_path])

# Evaluate if BERT model performs better with GloVe embeddings
from gensim.models import KeyedVectors

# Load GloVe embeddings
glove_path = "content/glove.6B.50d.txt"
glove_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

# Vectorize contexts and questions using GloVe
context_vectors_glove = [np.mean([glove_vectors[word] for word in context.split() if word in glove_vectors], axis=0) for context in contexts_sample]
question_vectors_glove = [np.mean([glove_vectors[word] for word in question.split() if word in glove_vectors], axis=0) for question in questions_sample]

# Combine context and question vectors
features_glove = [np.concatenate((context_vector, question_vector)) for context_vector, question_vector in zip(context_vectors_glove, question_vectors_glove)]

# Split the data into training and validation sets
X_train_glove, X_val_glove, y_train_glove, y_val_glove = train_test_split(features_glove, answers_sample, test_size=0.2, random_state=42)

# Print the shapes of the training and validation sets
print("Training set size with GloVe:", len(X_train_glove))
print("Validation set size with GloVe:", len(X_val_glove))

evaluate-v2.0.py already exists.
Training set size with GloVe: 80
Validation set size with GloVe: 20


# Evaluate BERT Model with GloVe Embeddings
Evaluate if the BERT model performs better when GloVe embeddings are used to embed the words.

In [15]:
# Evaluate if BERT model performs better with GloVe embeddings

from gensim.models import KeyedVectors

# Load GloVe embeddings
glove_path = "content/glove.6B.50d.txt"
glove_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

# Vectorize contexts and questions using GloVe
context_vectors_glove = [np.mean([glove_vectors[word] for word in context.split() if word in glove_vectors], axis=0) for context in contexts_sample]
question_vectors_glove = [np.mean([glove_vectors[word] for word in question.split() if word in glove_vectors], axis=0) for question in questions_sample]

# Combine context and question vectors
features_glove = [np.concatenate((context_vector, question_vector)) for context_vector, question_vector in zip(context_vectors_glove, question_vectors_glove)]

# Split the data into training and validation sets
X_train_glove, X_val_glove, y_train_glove, y_val_glove = train_test_split(features_glove, answers_sample, test_size=0.2, random_state=42)

# Print the shapes of the training and validation sets
print("Training set size with GloVe:", len(X_train_glove))
print("Validation set size with GloVe:", len(X_val_glove))

# Fine-tune BERT Model with GloVe embeddings

# Tokenize the input data
input_ids_glove = []
attention_masks_glove = []

for context, question in zip(contexts_sample, questions_sample):
    encoded_dict = tokenizer.encode_plus(
        question, context,
        add_special_tokens=True,
        max_length=512,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids_glove.append(encoded_dict['input_ids'])
    attention_masks_glove.append(encoded_dict['attention_mask'])

# Convert lists to tensors
input_ids_glove = torch.cat(input_ids_glove, dim=0)
attention_masks_glove = torch.cat(attention_masks_glove, dim=0)

# Convert answers to tensors
answer_starts_glove = [context.find(answer) for context, answer in zip(contexts_sample, answers_sample)]
answer_ends_glove = [start + len(answer) for start, answer in zip(answer_starts_glove, answers_sample)]
answer_starts_glove = torch.tensor(answer_starts_glove)
answer_ends_glove = torch.tensor(answer_ends_glove)

# Create DataLoader for training and validation sets
dataset_glove = TensorDataset(input_ids_glove, attention_masks_glove, answer_starts_glove, answer_ends_glove)
train_size_glove = int(0.8 * len(dataset_glove))
val_size_glove = len(dataset_glove) - train_size_glove
train_dataset_glove, val_dataset_glove = torch.utils.data.random_split(dataset_glove, [train_size_glove, val_size_glove])

train_dataloader_glove = DataLoader(train_dataset_glove, batch_size=8, shuffle=True)
val_dataloader_glove = DataLoader(val_dataset_glove, batch_size=8, shuffle=False)

# Load the BERT model for question answering
model_glove = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
model_glove.train()

# Set up the optimizer
optimizer_glove = AdamW(model_glove.parameters(), lr=2e-5)

# Training loop
epochs_glove = 3
for epoch in range(epochs_glove):
    total_loss_glove = 0
    for batch in train_dataloader_glove:
        b_input_ids, b_attention_masks, b_start_positions, b_end_positions = batch

        model_glove.zero_grad()

        outputs = model_glove(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_attention_masks,
            start_positions=b_start_positions,
            end_positions=b_end_positions
        )

        loss = outputs.loss
        total_loss_glove += loss.item()

        loss.backward()
        optimizer_glove.step()

    avg_train_loss_glove = total_loss_glove / len(train_dataloader_glove)
    print(f"Epoch {epoch + 1}, Loss with GloVe: {avg_train_loss_glove}")

# Save the fine-tuned model with GloVe embeddings
model_glove.save_pretrained('./fine_tuned_bert_glove')
tokenizer.save_pretrained('./fine_tuned_bert_glove')

# Print answers for 5 context and questions using the model fine-tuned with GloVe embeddings
model_glove.eval()
for i in range(5):
    context = contexts_sample[i]
    question = questions_sample[i]

    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = model_glove(input_ids, attention_mask=attention_mask)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    answer = ' '.join(all_tokens[torch.argmax(start_scores): torch.argmax(end_scores) + 1])

    print(f"Context: {context}")
    print(f"Question: {question}")
    print(f"Answer with GloVe: {answer}")
    print()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Training set size with GloVe: 80
Validation set size with GloVe: 20


C:\Users\Ganesh AI\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Ganesh AI\AppData\Roaming\Python\Python312\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated 

Epoch 1, Loss with GloVe: 6.2506085395812985
Epoch 2, Loss with GloVe: 6.035017728805542
Epoch 3, Loss with GloVe: 5.842369651794433
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Question: When did Beyonce start becoming popular?
Answer with GloVe: start becoming popular ? [SEP] beyonce gi ##selle knowles - carter ( / bi ##ː ##ˈ ##j ##ɒ ##n

In [16]:
# Print the comparison results
print(f"BERT Model Validation Loss: {avg_train_loss:.4f}")
print(f"Combined BERT Model with GloVe Validation Loss: {avg_train_loss_glove:.4f}")

# Draw conclusions
if avg_train_loss_glove < avg_train_loss:
    print("The Combined BERT model with GloVe embeddings performs better than the BERT model without GloVe embeddings.")
else:
    print("The BERT model without GloVe embeddings performs better than the Combined BERT model with GloVe embeddings.")

BERT Model Validation Loss: 5.7748
Combined BERT Model with GloVe Validation Loss: 5.8424
The BERT model without GloVe embeddings performs better than the Combined BERT model with GloVe embeddings.


## Conclusion
Based on the validation losses of both models, we can determine which model performs better. If the validation loss of the combined BERT model with GloVe embeddings is lower than that of the BERT model without GloVe embeddings, it indicates that the combined model performs better. Otherwise, the BERT model without GloVe embeddings performs better.